In [1]:
%config Completer.use_jedi = False

In [2]:
#!pip install statsmodels

In [3]:
import pandas as pd
import pickle
# import pickle5 as pickle
from tqdm.notebook import tqdm
import numpy as np
import statsmodels.api as sm
from libs.OLSNormModel import OLSNormModel

In [4]:
def save(x, file_name):
    with open(file_name, 'wb') as handle:
        pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)

def restore(file_name):
    with open(file_name, 'rb') as handle:
        x = pickle.load(handle)
    return x

In [5]:
df_ols = pd.read_pickle('../../data/df_ols.p')

# OLS

In [33]:
df_ols = df_ols.drop(columns='avg_citations_Q10')

In [34]:
df_ols.avg_citations_Q.value_counts()

low       357166
medium    316567
high      273783
Name: avg_citations_Q, dtype: int64

In [35]:
avg_citations_Q_dict = {"(0.1, 1.47)" : 'low', "(1.48, 1.74)" : 'medium', "(1.77, 4.07)": 'high'}
df_ols=df_ols.replace({"avg_citations_Q": avg_citations_Q_dict})

In [36]:
df_ols.avg_citations_Q.value_counts()

low       357166
medium    316567
high      273783
Name: avg_citations_Q, dtype: int64

In [37]:
## Now we use 'FIR_2','cit_rel_all_IAC' for the unnormalized versions

## normalized dep variable by topic

#I normalized the citations by their mean, so they are relative impact, and can be compared with the topic normalized
#df_ols['Cit_2_iac'] = df_ols.Cit_2_iac/df_ols.Cit_2_iac.mean()
#df_ols['FI_2'] = df_ols.FI_2/df_ols.FI_2.mean()

In [38]:
ols_norm = OLSNormModel(df_ols)

In [39]:
models = ['usnr_rank_cat','selindex','avg_citations_Q','avg_citations']
dep_vars = ['norm_cit_all_IAC','norm_FI_2','FIR_2','cit_rel_all_IAC']

In [40]:
fitted_models_norm = {}
for model in tqdm(models):
    for dv in tqdm(dep_vars):
        _ = ols_norm.main(model=model, dep = dv)
        fitted_models_norm[model+'+'+dv]=_

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
params_table_norm = ols_norm.get_params_table(fitted_models_norm)

/home/jovyan/rg_institutions/code/4_citations/libs/OLSNormModel.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(_)
/home/jovyan/rg_institutions/code/4_citations/libs/OLSNormModel.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(_)
/home/jovyan/rg_institutions/code/4_citations/libs/OLSNormModel.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(_)
/home/jovyan/rg_institutions/code/4_citations/libs/OLSNormModel.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(_)
/home/jovyan/rg_institutions/code/4_citations/libs/OLSNormModel.

In [42]:
params_table_norm.variable.value_counts()

Intercept                  16
white_F                    16
asian_F                    16
nb_auteur                  16
hispanic_F                 16
black_F                    16
asian_M                    16
black_M                    16
hispanic_M                 16
career_age                 16
usnr_rank_cat_top_10        4
usnr_rank_cat_top_100       4
selindex_more_selective     4
selindex_selective          4
avg_citations_Q_high        4
avg_citations_Q_medium      4
avg_citations               4
Name: variable, dtype: int64

In [43]:
params_table_norm[(params_table_norm.variable=='hispanic_F') & (params_table_norm.dep.str.contains('cit'))]

,variable,coeff,pvals,conf_lower,conf_higher,institution_cov,dep,n
7,hispanic_F,-0.129009,1.301866e-09,-0.170685,-0.087334,usnr_rank_cat,norm_cit_all_IAC,947516.0
7,hispanic_F,-0.231397,2.699222e-09,-0.307633,-0.155160,usnr_rank_cat,cit_rel_all_IAC,947516.0
7,hispanic_F,-0.143937,1.454251e-11,-0.185716,-0.102158,selindex,norm_cit_all_IAC,947516.0
7,hispanic_F,-0.252638,8.826567e-11,-0.328983,-0.176293,selindex,cit_rel_all_IAC,947516.0
7,hispanic_F,-0.135651,1.737565e-10,-0.177304,-0.093998,avg_citations_Q,norm_cit_all_IAC,947516.0
7,hispanic_F,-0.240639,6.072629e-10,-0.316851,-0.164427,avg_citations_Q,cit_rel_all_IAC,947516.0
7,hispanic_F,-0.133005,3.732839e-10,-0.174616,-0.091394,avg_citations,norm_cit_all_IAC,947516.0
7,hispanic_F,-0.236918,1.082202e-09,-0.313081,-0.160756,avg_citations,cit_rel_all_IAC,947516.0


In [44]:
params_table_norm.to_csv('../../results/linear_models/agg_model/ols_params.csv')